# Exploratory Data Analysis

---

1. Import packages
2. Loading data with Pandas
3. Descriptive statistics of data
4. Data visualization

---

## 1. Import packages

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

pd.options.mode.chained_assignment = None  # default='warn'

# Shows plots in jupyter notebook
%matplotlib inline


---

## 2. Loading data with Pandas

We need to load `historical_data.csv` into dataframe so that we can work with it in Python

In [77]:
path = "../data/historical_data.csv"
raw_data = pd.read_csv(path)

In [78]:
raw_data.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


We will have the possibility the predict the delivery time because all the feature needed are present in the dataset. 

---

## 3. Descriptive statistics of data

### Data types

It is useful to first understand the data that you're dealing with along with the data types of each column. The data types may dictate how you transform and engineer features.

In [79]:
raw_data.describe()

,market_id,store_id,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
count,196441.000000,197428.000000,196433.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,181166.000000,181166.000000,181166.000000,197428.000000,196902.000000
mean,2.978706,3530.510272,2.882352,3.196391,2682.331402,2.670791,686.218470,1159.588630,44.808093,41.739747,58.050065,308.560179,545.358935
std,1.524867,2053.496711,1.503771,2.666546,1823.093688,1.630255,522.038648,558.411377,34.526783,32.145733,52.661830,90.139653,219.352902
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,-86.000000,0.000000,-4.000000,-5.000000,-6.000000,0.000000,0.000000
25%,2.000000,1686.000000,1.000000,2.000000,1400.000000,1.000000,299.000000,800.000000,17.000000,15.000000,17.000000,251.000000,382.000000
50%,3.000000,3592.000000,3.000000,3.000000,2200.000000,2.000000,595.000000,1095.000000,37.000000,34.000000,41.000000,251.000000,544.000000
75%,4.000000,5299.000000,4.000000,4.000000,3395.000000,3.000000,949.000000,1395.000000,65.000000,62.000000,85.000000,446.000000,702.000000
max,6.000000,6987.000000,7.000000,411.000000,27100.000000,20.000000,14700.000000,14700.000000,171.000000,154.000000,285.000000,2715.000000,2088.000000


The describe method gives us a lot of information about the client data. The key point to take away from this is that we have highly skewed data, as exhibited by the percentile values.
We can see outliers that "maybe" we'll have to delete from the dataset. 

---

## 3. Feature engineering

---

### Deleting missing values

### Overview 
1. 16 columns / 13 numeric col + 3 cat col 
2. 197428 raws 
3. missing values 

In [80]:
# Deleting missing datas 
historical_df = raw_data.dropna()

### Convert columns to datetime
1. convert created_at
2. convert actual_delivery_time
3. create 1 column = différence between the 2 cols above

In [81]:
from datetime import datetime
historical_df["created_at"] = pd.to_datetime(historical_df["created_at"])
historical_df["actual_delivery_time"] = pd.to_datetime(historical_df["actual_delivery_time"])
historical_df["delivery_time_duration"]= (historical_df["actual_delivery_time"]- historical_df["created_at"]).dt.total_seconds()

### New column 
1. Minimum duration
2. preparation time
3. available ratio dasher

In [82]:
historical_df["minimum_duration"] = historical_df["estimated_order_place_duration"] + historical_df["estimated_store_to_consumer_driving_duration"]
historical_df["preparation_time"] = historical_df["delivery_time_duration"] - historical_df["minimum_duration"]
historical_df["busy_ratio_dasher"] = historical_df["total_busy_dashers"]/historical_df["total_onshift_dashers"]
historical_df["mean_price_per_order"] = historical_df["subtotal"]/historical_df["total_items"]
historical_df["ratio_distinct_item"] = historical_df["num_distinct_items"]/historical_df["total_items"]
historical_df["range_items_price"] = historical_df["max_item_price"]-historical_df["min_item_price"]


### Get dummies 
1. market_id
3. store_primary_category
4. order_protocol



In [83]:
## build function
def getdummies_df(df,col):
    df_dummies = pd.get_dummies(df[col]).add_prefix(f"{col}_")
    return df_dummies


In [84]:
market_id_dummies = getdummies_df(historical_df, "market_id" )
order_protocol_dummies = getdummies_df(historical_df, "order_protocol" )
store_primary_category_dummies = getdummies_df(historical_df, "store_primary_category" )


### Drop Unnecessary Columns 
1. drop create_at
2. drop actual_delivery_time
3. drop store_id
4. drop market_id
5. store_primary_category

In [85]:
historical_df_cleaned = historical_df.drop(["market_id",
                                            "created_at", 
                                            "actual_delivery_time",
                                            "store_primary_category",
                                            "store_id",
                                            "order_protocol",
                                            "max_item_price",
                                            "min_item_price",
                                            "subtotal",
                                            "total_items",
                                            "estimated_order_place_duration",
                                            "estimated_store_to_consumer_driving_duration",
                                            "minimum_duration",
                                            "total_busy_dashers",
                                            "total_onshift_dashers",],
                                            axis=1)

In [86]:
historical_df_cleaned

,num_distinct_items,total_outstanding_orders,delivery_time_duration,preparation_time,busy_ratio_dasher,mean_price_per_order,ratio_distinct_item,range_items_price
0,4,21.0,3779.0,2472.0,0.424242,860.250000,1.000000,682
1,1,2.0,4024.0,2888.0,2.000000,1900.000000,1.000000,0
8,3,18.0,1586.0,851.0,0.750000,1192.750000,0.750000,784
14,1,8.0,2273.0,1032.0,1.200000,1525.000000,1.000000,0
15,2,7.0,2988.0,2337.0,1.000000,1810.000000,1.000000,770
...,...,...,...,...,...,...,...,...
197423,3,23.0,3907.0,3325.0,1.000000,463.000000,1.000000,304
197424,4,14.0,3383.0,2217.0,0.916667,501.666667,0.666667,420
197425,3,40.0,3008.0,1962.0,1.051282,367.200000,0.600000,99
197426,1,12.0,3907.0,3077.0,1.000000,1175.000000,1.000000,0


### Concat historical_data_cleaned and df_dummies 
1. market_id_dummies
2. order_protocol_dummies
3. store_primary_category_dummies


In [87]:
historical_df_merge = pd.concat([historical_df_cleaned, 
                                 order_protocol_dummies,
                                 market_id_dummies,
                                 store_primary_category_dummies
                                 ], axis=1)
# align dtype over dataset
historical_df_merge = historical_df_merge.astype("float32")
historical_df_merge.head()

,num_distinct_items,total_outstanding_orders,delivery_time_duration,preparation_time,busy_ratio_dasher,mean_price_per_order,ratio_distinct_item,range_items_price,order_protocol_1.0,order_protocol_2.0,...,store_primary_category_southern,store_primary_category_spanish,store_primary_category_steak,store_primary_category_sushi,store_primary_category_tapas,store_primary_category_thai,store_primary_category_turkish,store_primary_category_vegan,store_primary_category_vegetarian,store_primary_category_vietnamese
0,4.0,21.0,3779.0,2472.0,0.424242,860.25,1.00,682.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2.0,4024.0,2888.0,2.000000,1900.00,1.00,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3.0,18.0,1586.0,851.0,0.750000,1192.75,0.75,784.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,1.0,8.0,2273.0,1032.0,1.200000,1525.00,1.00,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,2.0,7.0,2988.0,2337.0,1.000000,1810.00,1.00,770.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Multicolinearity
1. The Variance Inflation Factor (VIF) is a diagnostic tool used to detect multicollinearity in a regression model. VIF measures the degree of collinearity between each predictor and the other predictors in the model. Specifically, the VIF of a predictor quantifies how much the variance of the estimated regression coefficient is inflated due to collinearity with other predictors.

2. A high VIF value (typically greater than 5 or 10) suggests that a predictor is highly correlated with other predictors, indicating potential multicollinearity

In [94]:
### Separate target from features
y = historical_df_merge["delivery_time_duration"]
X = historical_df_merge.drop("delivery_time_duration", axis=1)

In [95]:
X.describe()

/Users/karimgheddache/Projets/projets_consulting/doordash/delivery-duration-prediction/consulting/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,num_distinct_items,total_outstanding_orders,preparation_time,busy_ratio_dasher,mean_price_per_order,ratio_distinct_item,range_items_price,order_protocol_1.0,order_protocol_2.0,order_protocol_3.0,...,store_primary_category_southern,store_primary_category_spanish,store_primary_category_steak,store_primary_category_sushi,store_primary_category_tapas,store_primary_category_thai,store_primary_category_turkish,store_primary_category_vegan,store_primary_category_vegetarian,store_primary_category_vietnamese
count,175777.000000,175777.000000,175777.000000,1.722740e+05,175777.000000,175777.000000,175777.000000,175777.000000,175777.000000,175777.000000,...,175777.000000,175777.000000,175777.000000,175777.000000,175777.000000,175777.000000,175777.000000,175777.000000,175777.000000,175777.000000
mean,2.675060,58.230114,2012.527466,NaN,975.351807,0.902036,475.193176,0.275372,0.118844,0.268095,...,0.000233,0.000142,0.005871,0.011964,0.000711,0.035471,0.001252,0.001320,0.004295,0.031688
std,1.625681,52.731041,1636.324951,NaN,515.608948,0.181600,518.680176,0.446703,0.323605,0.442969,...,0.015271,0.011925,0.076398,0.108724,0.026658,0.184968,0.035356,0.036306,0.065397,0.175168
min,1.000000,-6.000000,-670.000000,-inf,0.000000,0.012165,-197.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,17.000000,1275.000000,8.281250e-01,648.750000,0.833333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,41.000000,1803.000000,9.629630e-01,895.000000,1.000000,370.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,85.000000,2498.000000,1.000000e+00,1195.000000,1.000000,750.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,20.000000,285.000000,373152.000000,inf,14700.000000,1.000000,8400.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [90]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

def compute_vif(features):
    vif_data = pd.DataFrame()
    vif_data["feature"] = features
    vif_data["VIF"] = [variance_inflation_factor(historical_df_cleaned[features].values, i) for i in range(len(features))]
    return vif_data.sort_values(by=['VIF']).reset_index(drop=True)

In [92]:
### check infinite values
np.where(np.any(~np.isfinite(historical_df_cleaned), axis=0)==True)
## replace inf values with nan to drop all nan
historical_df_cleaned.replace([np.inf, -np.inf], np.)

(array([4]),)

KeyError: 4

In [91]:
# apply VIF computation to all columns
features = historical_df_cleaned.drop(columns=["delivery_time_duration"], axis=1).columns.to_list()
vif_data = compute_vif(features)
vif_data

MissingDataError: exog contains inf or nans